In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from numpy import prod

conf=SparkConf()
conf.set("spark.executor.memory", "1g")
conf.set("spark.cores.max", "2")
conf.setAppName("first_stream")
sc = SparkContext('local[2]', conf=conf)
from pyspark.streaming import StreamingContext

#............................................................................
##   Streaming with TCP/IP data
#............................................................................

#Create streaming context with latency of 1
streamContext = StreamingContext(sc,1)

lines = streamContext.socketTextStream("localhost", 9000)

def my_processing(rdd):
    a=rdd.map(lambda row:row.split(',')).map(lambda x:x[1])
    print(rdd.collect())
    print(a.collect())
    
lines.foreachRDD(my_processing)

streamContext.start()


